In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

C:\Users\IfanY\anaconda3\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\IfanY\anaconda3\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [2]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=False,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=False,
    transform=ToTensor(),
)

In [3]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y:{y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y:torch.Size([64]) torch.int64


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-31)

In [7]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [8]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_baches = len(dataloader)
    model.eval()
    test_loss, current = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            current += (pred.argmax(1)==y).type(torch.float).sum().item()
    test_loss /= num_baches
    current /= size
    print(f"Test Error: \n Accuracy: {(100 * current):>0.1f}%, Avg loss: {test_loss:>8f}\n")

In [9]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t + 1}\n-------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------
loss: 2.313850  [    0/60000]
loss: 2.309516  [ 6400/60000]
loss: 2.310773  [12800/60000]
loss: 2.303579  [19200/60000]
loss: 2.308517  [25600/60000]
loss: 2.309536  [32000/60000]
loss: 2.299196  [38400/60000]
loss: 2.302678  [44800/60000]
loss: 2.307226  [51200/60000]
loss: 2.293816  [57600/60000]
Test Error: 
 Accuracy: 9.4%, Avg loss: 2.305494

Epoch 2
-------------------
loss: 2.313850  [    0/60000]
loss: 2.309516  [ 6400/60000]
loss: 2.310773  [12800/60000]
loss: 2.303579  [19200/60000]
loss: 2.308517  [25600/60000]
loss: 2.309536  [32000/60000]
loss: 2.299196  [38400/60000]
loss: 2.302678  [44800/60000]
loss: 2.307226  [51200/60000]
loss: 2.293816  [57600/60000]
Test Error: 
 Accuracy: 9.4%, Avg loss: 2.305494

Epoch 3
-------------------
loss: 2.313850  [    0/60000]
loss: 2.309516  [ 6400/60000]
loss: 2.310773  [12800/60000]
loss: 2.303579  [19200/60000]
loss: 2.308517  [25600/60000]
loss: 2.309536  [32000/60000]
loss: 2.299196  [38400/60000]
loss: 

In [10]:
torch.save(model.state_dict(), "model.pth")
print("Saved Pytorch Model State to model.pth")

Saved Pytorch Model State to model.pth
